In [1]:
import torch
torch.cuda.empty_cache()


generate extractor

In [2]:
import sys
import os

# Add the parent directory to the path (adjust if your structure differs)
project_root = r"D:\Minor Project Ayan"
sys.path.append(project_root)

# Check it's added correctly
print("✅ Path added:", project_root)

# Now import the module
from utils.c3d_feature_extractor import extract_c3d_features


video_path = r"D:\Minor Project Ayan\data\Abuse001_x264.mp4"

# Save as .npy
extract_c3d_features(video_path,
                    save_path=r"D:\Minor Project Ayan\data\C3D_Features\Abuse001_features.npy",
                    use_cuda=True,
                    save_format="npy")

# Save as .mat
extract_c3d_features(video_path,
                    save_path=r"D:\Minor Project Ayan\data\C3D_Features\Abuse001_features.mat",
                    use_cuda=True,
                    save_format="mat")


✅ Path added: D:\Minor Project Ayan


c:\Users\rachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


array([[1.5702013 , 0.8906616 , 0.40881008, ..., 0.54290503, 0.3509516 ,
        0.57402194],
       [1.5678593 , 0.87527925, 0.42430186, ..., 0.36986795, 0.39370158,
        0.33279702],
       [1.2369174 , 0.6940583 , 0.46659434, ..., 0.30672607, 0.49007007,
        0.34028953],
       ...,
       [1.6662837 , 0.84106517, 0.4418934 , ..., 0.5299991 , 0.3423527 ,
        0.532659  ],
       [1.4092584 , 0.89826334, 0.34626147, ..., 0.39969313, 0.44348124,
        0.37054908],
       [1.5877883 , 0.77838045, 0.4565112 , ..., 0.3411828 , 0.45100173,
        0.2755862 ]], dtype=float32)

feature extraction

In [3]:
import os
import torch
import numpy as np
from utils.c3d_feature_extractor import extract_c3d_features

# Path to your dataset
dataset_path = r"D:\Minor Project Ayan\dataset_Extras"  # Update with your dataset path

# Subfolder names for each class
subfolders = ["Stealing","Vandalism"]

# Define the path where the extracted features will be saved
output_path = r"D:\Minor Project Ayan\data\C3D_Features"

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Process all videos in the dataset
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    print(f"Processing videos from '{subfolder}'...")

    # Iterate over all videos in the subfolder
    for video_name in os.listdir(subfolder_path):
        video_path = os.path.join(subfolder_path, video_name)
        torch.cuda.empty_cache()
        # Ensure the file is a video (check file extension)
        if video_path.endswith(('.mp4', '.avi', '.mov')):
            print(f"Processing video: {video_name}")
            
            # Define the path to save extracted features
            save_path = os.path.join(output_path, f"{subfolder}_{video_name.split('.')[0]}_features.npy")
            
            # Extract C3D features and save them
            extract_c3d_features(video_path, save_path=save_path, use_cuda=True, save_format="npy")

print("Feature extraction completed for all videos.")


Processing videos from 'Stealing'...
Processing video: Stealing002_x264.mp4
Processing video: Stealing003_x264.mp4
Processing video: Stealing004_x264.mp4
Processing video: Stealing006_x264.mp4
Processing video: Stealing007_x264.mp4
Processing video: Stealing008_x264.mp4
Processing video: Stealing009_x264.mp4
Processing video: Stealing010_x264.mp4
Processing video: Stealing011_x264.mp4
Processing video: Stealing012_x264.mp4
Processing video: Stealing013_x264.mp4
Processing video: Stealing014_x264.mp4
Processing video: Stealing015_x264.mp4
Processing video: Stealing016_x264.mp4
Processing video: Stealing017_x264.mp4
Processing video: Stealing018_x264.mp4
Processing video: Stealing019_x264.mp4
Processing video: Stealing020_x264.mp4
Processing video: Stealing021_x264.mp4
Processing video: Stealing022_x264.mp4
Processing video: Stealing023_x264.mp4
Processing video: Stealing024_x264.mp4
Processing video: Stealing025_x264.mp4
Processing video: Stealing026_x264.mp4
Processing video: Stealing0

prepare train data {x_Train.npy and y_Train_npy}

In [ ]:
import os
import numpy as np

# Directory where extracted features are saved
features_dir = r"D:\Minor Project Ayan\data\C3D_Features"

# Output files
features_output = r"D:\Minor Project Ayan\data\X_train.npy"
labels_output = r"D:\Minor Project Ayan\data\y_train.npy"

class_mapping = {
    "Abuse": 0,
    "Arrest": 1,
    "Arson": 2,
    "Assault": 3,
    "Burglary": 4,
    "Explosion": 5,
    "Fighting": 6,
    "Normal": 7,
    "RoadAccidents": 8,
    "Robbery": 9,
    "Shooting": 10,
    "Stealing": 11,
    "Vandalism": 12
}

X_train = []
y_train = []

# Loop over all .npy feature files
for file_name in os.listdir(features_dir):
    if not file_name.endswith("_features.npy"):
        continue

    # Example: 'Abuse_Abuse001_x264_features.npy' → 'Abuse'
    label_name = file_name.split("_")[0]
    label = class_mapping.get(label_name)

    if label is None:
        print(f"[WARNING] Skipping unknown class in file: {file_name}")
        continue

    feature_path = os.path.join(features_dir, file_name)
    features = np.load(feature_path)
    X_train.append(features)
    y_train.append(label)

# Convert to numpy arrays
X_train = np.array(X_train, dtype=object)  # Keep object dtype for variable-length segments
y_train = np.array(y_train)

# Save them
np.save(features_output, X_train)
np.save(labels_output, y_train)

print(f"[INFO] Saved {len(X_train)} feature arrays and labels.")


[WARNING] Skipping unknown class in file: Abuse001_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting001_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting004_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting005_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting006_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting007_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting009_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting010_x264_features.npy
[WARNING] Skipping unknown class in file: Shoplifting_Shoplifting013_x264_features.npy
[INFO] Saved 597 feature arrays and labels.


training model

In [7]:
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# Load the data
X_train = np.load('D:/Minor Project Ayan/data/X_train.npy', allow_pickle=True)
y_train = np.load('D:/Minor Project Ayan/data/y_train.npy', allow_pickle=True)

# Convert to proper dtype
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int64)

print(f"X_train dtype: {X_train.dtype}, shape: {X_train.shape}")
print(f"y_train dtype: {y_train.dtype}, shape: {y_train.shape}")
print(f"First element in X_train: {X_train[0]}")
print(f"First element in y_train: {y_train[0]}")

# Define model architecture
model = Sequential([
    LSTM(128, input_shape=(32, 512), return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

# Callbacks
checkpoint_dir = "D:/Minor Project Ayan/models/"
os.makedirs(checkpoint_dir, exist_ok=True)

callbacks = [
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    TensorBoard(
        log_dir=os.path.join(checkpoint_dir, 'logs'),
        histogram_freq=1
    )
]

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=60,
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

# Save final model
model.save(os.path.join(checkpoint_dir, 'final_model.keras'))
print("Model training complete and saved.")


X_train dtype: float32, shape: (597, 32, 512)
y_train dtype: int64, shape: (597,)
First element in X_train: [[1.5702013  0.8906616  0.40881008 ... 0.54290503 0.3509516  0.57402194]
 [1.5678593  0.87527925 0.42430186 ... 0.36986795 0.39370158 0.33279702]
 [1.2369174  0.6940583  0.46659434 ... 0.30672607 0.49007007 0.34028953]
 ...
 [1.6662837  0.84106517 0.4418934  ... 0.5299991  0.3423527  0.532659  ]
 [1.4092584  0.89826334 0.34626147 ... 0.39969313 0.44348124 0.37054908]
 [1.5877883  0.77838045 0.4565112  ... 0.3411828  0.45100173 0.2755862 ]]
First element in y_train: 0


c:\Users\rachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32, 128)        │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 381,825 (1.46 MB)

 Trainable params: 381,825 (1.46 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0894 - loss: -29.1110
Epoch 1: val_loss improved from inf to -330.33655, saving model to D:/Minor Project Ayan/models/best_model.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.0893 - loss: -30.2137 - val_accuracy: 0.0000e+00 - val_loss: -330.3365
Epoch 2/60
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0855 - loss: -186.6964
Epoch 2: val_loss improved from -330.33655 to -847.19043, saving model to D:/Minor Project Ayan/models/best_model.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0855 - loss: -187.5100 - val_accuracy: 0.0000e+00 - val_loss: -847.1904
Epoch 3/60
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0688 - loss: -445.6006
Epoch 3: val_loss improved from -847.19043 to -1662.23267, saving model to D:/Minor Project Ayan/models/best_model.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0693 - loss: -447.7639 - val_accuracy: 0.0000e+00 - val_loss: -1662.2327
Epoch

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Training Performance")
plt.show()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import model_from_json
from tensorflow.keras.models import load_model
import os

# Load C3D model
def load_c3d_model():
    model_json_path = "D:\Minor Project Ayan\models\model.json"
    model_weights_path = "D:\Minor Project Ayan\models\weights_L1L2.mat"

    with open(model_json_path, 'r') as json_file:
        c3d_model = model_from_json(json_file.read())

    import h5py
    import scipy.io
    mat = scipy.io.loadmat(model_weights_path)
    weights = mat['weights'][0]

    for layer, w in zip(c3d_model.layers, weights):
        if hasattr(layer, 'set_weights'):
            layer.set_weights([w[0], w[1]])

    return c3d_model

# Resize + preprocess frame
def preprocess_frame(frame):
    frame = cv2.resize(frame, (112, 112))
    frame = frame.astype(np.float32)
    frame -= 128
    return frame

# Extract 32 segments from the video and feed to C3D
def extract_c3d_features(video_path, c3d_model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = preprocess_frame(frame)
        frames.append(frame)
    cap.release()

    total_frames = len(frames)
    if total_frames < 16:
        raise ValueError("Video too short to extract C3D features.")

    # Split video into 32 equal segments (each of 16 frames)
    segment_size = total_frames // 32
    features = []

    for i in range(32):
        start = i * segment_size
        end = start + 16
        if end > total_frames:
            segment = frames[-16:]
        else:
            segment = frames[start:end]

        segment = np.array(segment)
        segment = np.expand_dims(segment, axis=0)  # shape: (1, 16, 112, 112, 3)
        feature = c3d_model.predict(segment)[0]
        features.append(feature)

    return np.array(features)  # shape: (32, 512)

# Inference using trained LSTM model
def predict_anomaly(features):
    lstm_model_path = "D:/Minor Project Ayan/models/best_model.keras"
    model = load_model(lstm_model_path)

    features = np.expand_dims(features, axis=0)  # shape: (1, 32, 512)
    prediction = model.predict(features)[0][0]

    label = "Anomalous" if prediction >= 0.5 else "Normal"
    return prediction, label

# Main function
def run_inference(video_path):
    print(f"Loading video: {video_path}")
    c3d_model = load_c3d_model()
    features = extract_c3d_features(video_path, c3d_model)
    score, label = predict_anomaly(features)
    print(f"Prediction Score: {score:.4f} → {label}")

# Run this to test
if __name__ == "__main__":
    video_path = "D:/Minor Project Ayan/data/Abuse001_x264.mp4"  # change as needed
    run_inference(video_path)
